# Montar o Google Drive

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
'Colab Notebooks'   images  'My Drive'	 navios   yolov4


# Clonar e buildar o darknet

In [3]:
%%capture
!git clone https://github.com/AlexeyAB/darknet
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make

# Carregar o dataset

In [4]:
%%capture
!cp /mydrive/navios/fold_1.zip ../
!cp /mydrive/navios/fold_2.zip ../
!cp /mydrive/navios/fold_3.zip ../
!cp /mydrive/navios/fold_4.zip ../
!cp /mydrive/navios/fold_5.zip ../
!cp /mydrive/navios/test.zip ../
!unzip ../fold_1.zip -d data/
!unzip ../fold_2.zip -d data/
!unzip ../fold_3.zip -d data/
!unzip ../fold_4.zip -d data/
!unzip ../fold_5.zip -d data/
!unzip ../test.zip -d data/

# Configurar o YOLOv4

In [5]:
# CFG
!cp /mydrive/navios/navios-train.cfg ./cfg
!cp /mydrive/navios/navios-nfold.cfg ./cfg
!cp /mydrive/navios/navios-test.cfg ./cfg

In [6]:
# obj.names e obj.data
!cp /mydrive/navios/train.names ./data
!cp /mydrive/navios/train.data  ./data
!cp /mydrive/navios/train_kfold_1.data  ./data
!cp /mydrive/navios/train_kfold_2.data  ./data
!cp /mydrive/navios/train_kfold_3.data  ./data
!cp /mydrive/navios/train_kfold_4.data  ./data
!cp /mydrive/navios/train_kfold_5.data  ./data
!cp /mydrive/navios/train_1fold.data  ./data
!cp /mydrive/navios/train_2fold.data  ./data
!cp /mydrive/navios/train_3fold.data  ./data
!cp /mydrive/navios/train_4fold.data  ./data
!cp /mydrive/navios/train_5fold.data  ./data


In [7]:
# test.txt
!cp /mydrive/navios/valid_kfold_1.txt ./data
!cp /mydrive/navios/valid_kfold_2.txt ./data
!cp /mydrive/navios/valid_kfold_3.txt ./data
!cp /mydrive/navios/valid_kfold_4.txt ./data
!cp /mydrive/navios/valid_kfold_5.txt ./data
!cp /mydrive/navios/test.txt ./data


In [11]:
!sed -i 's/\r//g' data/test.txt

# Funções úteis

In [8]:
# define helper functions
import cv2
import matplotlib.pyplot as plt
import os
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
%matplotlib inline

def imShow(path):
  image = cv2.imread(path)
  height, width = image.shape[:2]
  resized_image = cv2.resize(image,(3*width, 3*height), interpolation = cv2.INTER_CUBIC)

  fig = plt.gcf()
  fig.set_size_inches(18, 10)
  plt.axis("off")
  plt.imshow(cv2.cvtColor(resized_image, cv2.COLOR_BGR2RGB))
  plt.show()

# Resutados

## Resultados para o treinamento completo

In [ ]:
# Cálculo do map
!./darknet detector map data/train.data cfg/navios-train.cfg /mydrive/navios/backup_train/navios-train_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   

In [ ]:
# Teste
!./darknet detector test data/train.data cfg/navios-test.cfg /mydrive/navios/backup_train/navios-train_best.weights -ext_output -dont_show < data/test.txt > resultados_train.txt
!cp resultados_train.txt /mydrive/navios/

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   9 route  8 2 	                           ->  320 x 176 x 128 
  10 conv 

## Resultados para 1 fold

In [ ]:
# Cálculo do map
!./darknet detector map data/train_1fold.data cfg/navios-nfold.cfg /mydrive/navios/backup_1fold/navios-nfold_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   

In [ ]:
# Teste
!./darknet detector test data/train_1fold.data cfg/navios-nfold.cfg /mydrive/navios/backup_1fold/navios-nfold_best.weights -ext_output -dont_show < data/test.txt > resultados_1fold.txt
!cp resultados_1fold.txt /mydrive/navios/

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   9 route  8 2 	                           ->  320 x 176 x 128 
  10 conv 

## Resultados para 2 folds

In [ ]:
# Cálculo do map
!./darknet detector map data/train_2fold.data cfg/navios-nfold.cfg /mydrive/navios/backup_2fold/navios-nfold_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   

In [ ]:
# Teste
!./darknet detector test data/train_2fold.data cfg/navios-nfold.cfg /mydrive/navios/backup_2fold/navios-nfold_best.weights -ext_output -dont_show < data/test.txt > resultados_2fold.txt
!cp resultados_2fold.txt /mydrive/navios/

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   9 route  8 2 	                           ->  320 x 176 x 128 
  10 conv 

## Resultados para 3 folds

In [ ]:
# Cálculo do map
!./darknet detector map data/train_3fold.data cfg/navios-nfold.cfg /mydrive/navios/backup_3fold/navios-nfold_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   

In [ ]:
# Teste
!./darknet detector test data/train_3fold.data cfg/navios-nfold.cfg /mydrive/navios/backup_3fold/navios-nfold_best.weights -ext_output -dont_show < data/test.txt > resultados_3fold.txt
!cp resultados_3fold.txt /mydrive/navios/

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   9 route  8 2 	                           ->  320 x 176 x 128 
  10 conv 

## Resultados para 4 folds

In [ ]:
# Cálculo do map
!./darknet detector map data/train_4fold.data cfg/navios-nfold.cfg /mydrive/navios/backup_4fold/navios-nfold_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 4, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   

In [ ]:
# Teste
!./darknet detector test data/train_4fold.data cfg/navios-nfold.cfg /mydrive/navios/backup_4fold/navios-nfold_best.weights -ext_output -dont_show < data/test.txt > resultados_4fold.txt
!cp resultados_4fold.txt /mydrive/navios/

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   9 route  8 2 	                           ->  320 x 176 x 128 
  10 conv 

## Resultados para 5 fold

In [9]:
# Cálculo do map
!./darknet detector map data/train_5fold.data cfg/navios-nfold.cfg /mydrive/navios/backup_5fold/navios-nfold_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   

In [12]:
# Teste
!./darknet detector test data/train_5fold.data cfg/navios-nfold.cfg /mydrive/navios/backup_5fold/navios-nfold_best.weights -ext_output -dont_show < data/test.txt > resultados_5fold.txt
!cp resultados_5fold.txt /mydrive/navios/

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
   layer   filters  size/strd(dil)      input                output
   0 conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   9 route  8 2 	                           ->  320 x 176 x 128 
  10 conv 

## Resultados para validação no fold 1

In [ ]:
# Cálculo do map
!./darknet detector map data/train_kfold_1.data cfg/navios-train.cfg /mydrive/navios/backup_kfold_1/navios-train_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   

## Resultados para validação no fold 2

In [ ]:
# Cálculo do map
!./darknet detector map data/train_kfold_2.data cfg/navios-train.cfg /mydrive/navios/backup_kfold_2/navios-train_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   

## Resultados para validação no fold 3

In [ ]:
# Cálculo do map
!./darknet detector map data/train_kfold_3.data cfg/navios-train.cfg /mydrive/navios/backup_kfold_3/navios-train_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   

## Resultados para validação no fold 4

In [ ]:
# Cálculo do map
!./darknet detector map data/train_kfold_4.data cfg/navios-train.cfg /mydrive/navios/backup_kfold_4/navios-train_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   

## Resultados para validação no fold 5

In [ ]:
# Cálculo do map
!./darknet detector map data/train_kfold_5.data cfg/navios-train.cfg /mydrive/navios/backup_kfold_5/navios-train_best.weights -dont_show

 CUDA-version: 12020 (12020), cuDNN: 8.9.6, CUDNN_HALF=1, GPU count: 1  
 CUDNN_HALF=1 
 OpenCV version: 4.5.4
 0 : compute_capability = 750, cudnn_half = 1, GPU: Tesla T4 
net.optimized_memory = 0 
mini_batch = 1, batch = 8, time_steps = 1, train = 0 
   layer   filters  size/strd(dil)      input                output
   0 Create CUDA-stream - 0 
 Create cudnn-handle 0 
conv     32       3 x 3/ 1    640 x 352 x   3 ->  640 x 352 x  32 0.389 BF
   1 conv     64       3 x 3/ 2    640 x 352 x  32 ->  320 x 176 x  64 2.076 BF
   2 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   3 route  1 		                           ->  320 x 176 x  64 
   4 conv     64       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  64 0.461 BF
   5 conv     32       1 x 1/ 1    320 x 176 x  64 ->  320 x 176 x  32 0.231 BF
   6 conv     64       3 x 3/ 1    320 x 176 x  32 ->  320 x 176 x  64 2.076 BF
   7 Shortcut Layer: 4,  wt = 0, wn = 0, outputs: 320 x 176 x  64 0.004 BF
   8 conv   